In [ ]:
# lib
import numpy as np
import matplotlib.pyplot as plt
import copy
from typing import List, Dict, Optional
from icecream import ic

# custom lib:
import jx_lib 


In [ ]:
inputRGB = np.array([
        [255,0,0],
        [0,255,0],
        [0,0,255],
        [255,255,0],
        [255,0,255],
        [0,255,255],
        [128,128,0],
        [128,0,128],
        [0,128,128],
        [255,128,0],
        [255,0,128],
        [128,255,0],
        [0,255,128],
        [128,0,255],
        [0,128,255],
        [255,20,147],
        [220,20,60],
        [255,51,51],
        [255,153,51],
        [255,255,51],
        [51,255,51],
        [153,255,51],
        [51,255,153],
        [51,255,255]])
    

In [ ]:
ic(np.shape(inputRGB))

# normalization
normRGB = inputRGB/255
plt.imshow(np.reshape(normRGB,(normRGB.shape[0],1,3)))
ic(normRGB.shape)


In [ ]:

from IPython import display

class SOM:
    def __init__(
        self,
        training_data, # Assumed: normalized
        space: int      = 100, # 100 x 100 grid of neurons
        alpha_0: float  = 0.8,
        verbose: bool   = True,
        path: str       = "output",
    ):
        # Initialize the system
        self.training_data = training_data
        self.space = space
        self.alpha_0 = alpha_0
        self.verbose = verbose
        self.path = path

        # Initialize random weights
        self.w = np.random.random((space,space,3))

        # init output
        jx_lib.create_all_folders(path)
        if verbose:
            self.imshow(data=self.w, name="w_0")

    def imshow(self, data, name:str, save:bool=True):
        fig = plt.figure()
        plt.imshow(data)
        if save:
            fig.savefig("{}/{}.png".format(self.path, name), bbox_inches = 'tight')
        if self.verbose:
            display.display(plt.gcf())

    def learn(
        self,
        sigma_0: int = 10, # [10,40,70]
        tot_training_epochs: int = 10 #600
    ):
        self.tot_training_epochs = tot_training_epochs
        # learn:
        epoch = 1
        T = tot_training_epochs
        N = self.space
        while epoch <= tot_training_epochs:
            k = epoch
                
            # Alpha(k) & s(k):
            alpha_k = self.alpha_0 * np.exp(- k / T)
            s_k = sigma_0 * np.exp(- k / T)
            s_k_2_2_division = 1 / (2 * s_k ** 2) # pre-optimization
            # w_ij:
            for x in self.training_data:
                # calculate performance index
                diff = np.linalg.norm(x - self.w, axis =2)
                # find index of winning node
                ind = np.unravel_index(np.argmin(diff, axis=None), diff.shape)
                # Update weights for neighbourhood
                xx = np.arange(0, N, 1)
                yy = np.arange(0, N, 1)

                ### for loop:
                # for i in xx:
                #     for j in xx:
                #         dij = np.array(ind) - [i, j]
                #         dij2 = np.dot(dij, np.transpose(dij))
                #         Nijk = np.exp(- dij2 * s_k_2_2_division)
                #         self.w[i][j] = self.w[i][j] + alpha_k * Nijk * (x - self.w[i][j])

                ### matrix form:
                Mj = np.meshgrid(xx, yy)
                Dx = (Mj[0] - ind[0]) ** 2
                Dy = (Mj[1] - ind[1]) ** 2
                Dij2 = Dx+Dy
                Nij = np.exp(- Dij2 * s_k_2_2_division )
                dxw = np.subtract(x, self.w)
                Nw = np.stack([Nij, Nij, Nij], axis=2) # depth stacking
                self.w = self.w + alpha_k * np.multiply(Nw, dxw)

            plot_ind = [1, 20, 40, 100, 600]
            if epoch in plot_ind:
                print("Epoch Number: {}".format(epoch))
                self.imshow(data=self.w, 
                    name="[s={}]_w_{}".format(sigma_0, epoch))
            
            epoch += 1

In [ ]:
som = SOM(
    training_data = normRGB,
    # DEFAULT:
    space   = 100, # 100 x 100 grid of neurons
    alpha_0 = 0.8,
    verbose = True,
    path    = "output/p2",
)
som.learn(
    sigma_0 = 10, # [10,40,70]
    tot_training_epochs= 600
)

In [ ]:
w = np.ones((4,4,3))
ic(w)

x = np.array([0, 0, 0.1])
ic(x)

N = np.ones((3,3))*10
w = np.subtract(w,x)
ic(w)
N = np.array([[1., 2., 3., 4.],
              [1., 2., 3., 4.],
              [1., 2., 3., 4.],
              [2., 3., 4., 5.]])
ic(N)
Nw = np.stack([N, N, N], axis=2)
ic(Nw)
ic(np.shape(Nw))
ic(np.multiply(Nw, w))

In [ ]:
N = np.array([[1., 2., 3., 4.],
              [1., 2., 3., 4.],
              [1., 2., 3., 4.],
              [2., 3., 4., 5.]])
ic(N)
Nw = np.stack([N, N, N], axis=2)
ic(Nw)
ic(np.shape(Nw))
ic(np.multiply(Nw, w))